In [1]:
import pandas as pd
import requests
import folium
!conda install -c conda-forge beautifulsoup4 lxml --yes 
!conda install -c conda-forge geopy --yes
!conda install -c conda-forge geocoder --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - beautifulsoup4
    - lxml


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    soupsieve-1.9.4            |           py36_0          58 KB  conda-forge
    beautifulsoup4-4.8.1       |           py36_0         149 KB  conda-forge
    lxml-4.4.1                 |   py36h7ec2d77_0         1.6 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         1.8 MB

The following NEW packages will be INSTALLED:

    soupsieve:      1.9.4-py36_0         conda-forge

The following packages will be UPDATED:


In [2]:
from bs4 import BeautifulSoup
import json
import numpy as np
import urllib
url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

soup = BeautifulSoup(requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text, 'lxml')
data = soup.find('table',{'class':'wikitable sortable'})

df = pd.read_html(str(data), header = 0)[0]
df = pd.DataFrame(np.array(df), columns = ["PostalCode", "Borough", "Neighborhood"])
df = df[df.Borough != 'Not assigned']

df = df.groupby('PostalCode').agg({'Borough':'first',
                                  'Neighborhood': ', '.join,}).reset_index()
df['Neighborhood'] = np.where((df['Neighborhood'] == 'Not assigned'),
                               df['Borough'],
                               df['Neighborhood'])

df

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv..."
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."


In [3]:
df.shape

(103, 3)

In [4]:

from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [5]:
address = 'Toronto, TO'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto  are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto  are 43.653963, -79.387207.


In [6]:

import geocoder # import geocoder



In [9]:
url = 'http://cocl.us/Geospatial_data'
postal_code_data = pd.read_csv(url)
postal_code_data
postal_code_data.rename({'Postal Code': 'PostalCode'}, axis=1, inplace=True)
postal_code_data
tornto_data=df.merge(postal_code_data,on='PostalCode')
tornto_data
tornto_data

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv...",43.688905,-79.554724
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437


In [10]:
CLIENT_ID = 'ZNIGMWEGB3B2PRX1KKCUDR52MK2HBXMHHHXZUKK2D1KJDOIS' # your Foursquare ID
CLIENT_SECRET = 'FKC5BPFX5A3DLZJNDBC413YNIWXGWJI53FFEE3CFXO4MHENT' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
neighborhood_latitude = tornto_data.loc[0, 'Latitude']
neighborhood_longitude = tornto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = tornto_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))


Latitude and longitude values of Rouge, Malvern are 43.806686299999996, -79.19435340000001.
